# CrossGL: Revolutionizing Shader Development

## 🌟 The Universal Shader Language

In the ever-evolving world of graphics programming, **CrossGL** emerges as a game-changing solution, bridging the gap between diverse graphics APIs.

### 🚀 Write Once, Run Everywhere

Imagine writing a shader _once_ and deploying it across:

- 🍎 Metal
- 🎮 DirectX
- 🖥️ OpenGL

...all without changing a single line of code!

## 💡 Key Benefits

1. **⏱️ Time-Saving**: Slash development time by eliminating the need for multiple shader versions.
2. **🛠️ Consistency**: Ensure uniform behavior across all platforms.
3. **🧠 Simplified Learning Curve**: Master one language instead of many.
4. **🔍 Enhanced Debugging**: Develop universal tools for shader analysis.
5. **🔮 Future-Proof**: Easily adapt to new graphics APIs as they emerge.


📚 **Documentation**
Comprehensive documentation is available to help you get started and master CrossGL:

🔗 [CrossGL Documentation](https://crossgl.github.io/language.html)
📘 [Contribution Guidelines](https://crossgl.github.io/contribution.html)


🤝 **Community & Social Media**
Join our vibrant community of developers and graphics enthusiasts:

💬 [Discord Channel](https://discord.gg/mYH45zZ9)
- Get real-time support
- Share your projects
- Collaborate with other CrossGL users

🐙 [GitHub](https://github.com/CrossGL)
- Access the source code
- Report issues
- Contribute to the project

🐦 [X](https://x.com/crossGL_)
- Stay updated with the latest news
- Join discussions about graphics programming


👥 [LinkedIn](https://www.linkedin.com/company/crossgl/?viewAsMember=true)
- Connect with the CrossGL team
- Explore career opportunities
- Network with industry professionals


## 🌈 CrossGL Shader 

```cpp
shader main {
    vertex {
        input vec3 position;
        output vec2 vUV;

        void main()
        {
            vUV = position.xy * 10.0;
            gl_Position = vec4(position, 1.0);
        }
    }

    float perlinNoise(vec2 p) {
        return fract(sin(dot(p, vec2(12.9898, 78.233))) * 43758.5453);
    }

    fragment {
        input vec2 vUV;
        output vec4 fragColor;

        void main()
        {
            float noise = perlinNoise(vUV);
            float height = noise * 10.0;
            vec3 color = vec3(height / 10.0, 1.0 - height / 10.0, 0.0);
            fragColor = vec4(color, 1.0);
        }
    }
}
```

## 🤝 Unifying the Developer Community

CrossGL isn't just a language; it's a bridge connecting developers across different platforms, fostering collaboration and innovation.

### 📈 The CrossGL Advantage

| Feature | Without CrossGL | With CrossGL |
|---------|-----------------|--------------|
| Platforms Supported | Single | Multiple |
| Code Reusability | Low | High |
| Learning Curve | Steep | Gentle |
| Time to Market | Slower | Faster |

## 🎨 Unleash Your Creativity

With CrossGL, you're no longer bound by platform limitations. Your imagination is the only limit!

> "CrossGL is not just a tool; it's a canvas for your graphics ambitions." - Graphics Guru

---

Are you ready to transform your shader development workflow? Dive into CrossGL and experience the future of graphics programming today! 🚀✨


# CrossGL Translator: A Step-by-Step Journey

## 🚀 Embark on a Cross-Platform Shader Adventure

Let's walk through the magic of CrossGL translation, step by step!

### Step 1: The CrossGL Source 📝

We begin with a CrossGL shader that generates Perlin noise for terrain:

In [2]:
!wget -O test.cgl https://raw.githubusercontent.com/CrossGL/demos/main/samples/Basic%20Fog%20Shader.cgl

--2024-09-10 13:49:29--  https://raw.githubusercontent.com/CrossGL/demos/main/samples/Basic%20Fog%20Shader.cgl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8000::154, 2606:50c0:8001::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 725 [text/plain]
Saving to: ‘test.cgl’

test.cgl            100%[===================>]     725  --.-KB/s    in 0s      

2024-09-10 13:49:31 (30.4 MB/s) - ‘test.cgl’ saved [725/725]



In [3]:
with open("test.cgl",'r') as f:
    cgl_code = f.read()
print(cgl_code)

shader main {
    vertex {
        input vec3 position;
        input vec2 texCoord;
        output vec2 fragTexCoord;
        output vec3 fragPosition;

        void main() {
            fragTexCoord = texCoord;
            fragPosition = position;
            gl_Position = vec4(position, 1.0);
        }
    }

    fragment {
        input vec2 fragTexCoord;
        input vec3 fragPosition;
        output vec4 fragColor;

        void main() {
           
            float depth = length(fragPosition);
            float fogFactor = exp(-fogDensity * depth);
            vec3 finalColor = mix(fogColor, baseColor, fogFactor);
            fragColor = vec4(finalColor, 1.0);
        }
    }
}



### Step 2: Install Crosstl 🧩


In [6]:
! pip uninstall crosstl -y

Found existing installation: crosstl 0.0.1.3
Uninstalling crosstl-0.0.1.3:
  Successfully uninstalled crosstl-0.0.1.3


In [1]:
import crosstl

### Step 3: The Translation Magic ✨

Now, we unleash our code generators:

#### 3.1 Metal Transformation 🍎

In [8]:
metal_code = crosstl.translate('test.cgl', backend='metal', save_shader='output.metal')
print(metal_code)

#include <metal_stdlib>
using namespace metal;

struct Vertex_INPUT {
    float3 position [[attribute(0)]];
    float2 texCoord [[attribute(1)]];
};

struct Vertex_OUTPUT {
    float2 fragTexCoord;
    float3 fragPosition;
};


vertex Vertex_OUTPUT main(Vertex_INPUT input [[stage_in]]) {
    Vertex_OUTPUT output;
     output.fragTexCoord =  input.texCoord;
     output.fragPosition =  input.position;
     gl_Position = float4( input.position, 1.0);
    return output;
}

struct Fragment_INPUT {
    float2 fragTexCoord [[stage_in]];
    float3 fragPosition;
};

struct Fragment_OUTPUT {
    float4 fragColor [[color(0)]];
};


fragment Fragment_OUTPUT main(Fragment_INPUT input [[stage_in]]) {
    Fragment_OUTPUT output;
    float depth = length( input.fragPosition);
    float fogFactor = exp(- fogDensity *  depth);
    float3 finalColor = mix( fogColor,  baseColor,  fogFactor);
     output.fragColor = float4( finalColor, 1.0);
    return output;
}




#### 3.2 DirectX (HLSL) Transformation 🎮

In [9]:
hlsl_code = crosstl.translate('test.cgl', backend='directx',save_shader='output.hlsl')
print(hlsl_code)


struct VSInput {
    float3 position : POSITION;
    float2 texCoord : TEXCOORD0;
};

struct VSOutput {
    float2 fragTexCoord : TEXCOORD0;
    float3 fragPosition : TEXCOORD1;
};


VSOutput VSMain(VSInput input) {
    VSOutput output;
     output.fragTexCoord =  input.texCoord;
     output.fragPosition =  input.position;
     gl_Position = float4( input.position, 1.0);
    return output;
}

struct PSInput {
    float2 fragTexCoord : TEXCOORD0;
    float3 fragPosition : TEXCOORD1;
};

struct PSOutput {
    float4 fragColor : SV_TARGET0;
};


PSOutput PSMain(PSInput input) {
    PSOutput output;
    float depth = length( input.fragPosition);
    float fogFactor = exp(-  fogDensity *  depth);
    float3 finalColor = mix( fogColor,  baseColor,  fogFactor);
     output.fragColor = float4( finalColor, 1.0);
    return output;
}




#### 3.3 OpenGL (GLSL) Transformation 🖥️

In [10]:
glsl_code = crosstl.translate('test.cgl',backend='opengl', save_shader='output.glsl')
print(glsl_code)

#version 450


// Vertex shader

layout(location = 0) in vec3 position;
layout(location = 1) in vec2 texCoord;
out vec2 fragTexCoord;
layout(location = 0) out vec3 fragPosition;


void main() {
     fragTexCoord =  texCoord;
     fragPosition =  position;
     gl_Position = vec4( position, 1.0);
}


// Fragment shader

in vec2 fragTexCoord;
layout(location = 0) in vec3 fragPosition;
layout(location = 0) out vec4 fragColor;


void main() {
    float depth = length( fragPosition);
    float fogFactor = exp(- fogDensity *  depth);
    vec3 finalColor = mix( fogColor,  baseColor,  fogFactor);
     fragColor = vec4( finalColor, 1.0);
}




### Step 4: Marvel at the Results 🎉

From a single CrossGL source, we've generated shaders for three major graphics APIs!

## 🌟 The Power of CrossGL

This demo showcases the true potential of CrossGL:
- **One Source, Multiple Targets**: Write once, run everywhere.
- **Seamless Translation**: Automatic adaptation to platform-specific syntax and features.
- **Consistency Across Platforms**: Ensure your shaders behave identically on all supported backends.

By leveraging CrossGL, developers can focus on crafting amazing shaders without worrying about platform-specific intricacies. It's not just a translator; it's a bridge to unlimited creative possibilities in the world of graphics programming! 🚀🎨


## 🔄 Two-Way Translation: From Platform-Specific to CrossGL

CrossGL doesn't just translate from a universal language to platform-specific shaders - it also works in reverse! This powerful feature allows developers to convert existing shaders from various platforms into CrossGL.

### Step 5: Reverse Translation 🔀

Let's explore how we can convert shaders from Metal, DirectX, and OpenGL back into CrossGL.

#### 5.1 OpenGL (GLSL) to CrossGL 🖥️➡️🌐

In [11]:
glsl_crossgl_code = crosstl.translate('output.glsl', 'cgl')
print(glsl_crossgl_code)

shader main {
    vertex {
        input vec3 position;
        input vec2 texCoord;
        output vec3 fragPosition;
        output vec2 fragTexCoord;



        void main() {
         fragTexCoord =  texCoord;
         fragPosition =  position;
         gl_Position = vec4( position, 1.0);
        }
    }
    fragment {
        input vec3 fragPosition;
        output vec4 fragColor;
        input vec2 fragTexCoord;


        void main() {
        float depth = length( fragPosition);
        float fogFactor = exp(- fogDensity *  depth);
        vec3 finalColor = mix( fogColor,  baseColor,  fogFactor);
         fragColor = vec4( finalColor, 1.0);
        }
    }
}



### 5.2 DirectX (HLSL) to CrossGL 🎮➡️🌐

In [12]:
hlsl_crossgl_code = crosstl.translate('output.hlsl', backend='cgl')
print(hlsl_crossgl_code)

shader main {
    // Vertex Shader
    vertex {
        input vec3 position;
        input vec2 texCoord;
        output vec2 fragTexCoord;
        output vec3 fragPosition;
        void main() {
                        fragTexCoord = texCoord;
            fragPosition = position;
             gl_Position = vec4(position, 1.0);
                    }
    }

    // Fragment Shader
    fragment {
        input vec2 fragTexCoord;
        input vec3 fragPosition;
        output vec4 fragColor;
        void main() {
                        float depth = length(fragPosition);
            float fogFactor = exp(- fogDensity *  depth);
            vec3 finalColor = mix( fogColor,  baseColor,  fogFactor);
            fragColor = vec4( finalColor, 1.0);
                    }
    }
}



### 5.3 Metal to CrossGL 🍎➡️🌐

In [13]:
metal_crossgl_code = crosstl.translate('output.metal',save_shader='output.cgl', backend='cgl')
print(metal_crossgl_code)

shader main {
 
    // Vertex Shader
    vertex {
        input vec3 position;
        input vec2 texCoord;
        output vec2 fragTexCoord;
        output vec3 fragPosition;
        void main() {
                        fragTexCoord = texCoord;
            fragPosition = position;
             gl_Position = vec4(position, 1.0);
                    }
    }

    // Fragment Shader
    fragment {
        input vec2 fragTexCoord;
        input vec3 fragPosition;
        output vec4 fragColor;
        void main() {
                        float depth = length(fragPosition);
            float fogFactor = exp((- fogDensity) *  depth);
            vec3 finalColor = mix( fogColor,  baseColor,  fogFactor);
            fragColor = vec4( finalColor, 1.0);
                    }
    }
}



In [2]:
slang_crossgl_code = crosstl.translate('output.slang',save_shader='output.cgl', backend='cgl')
print(slang_crossgl_code)

shader main {
    struct AssembledVertex {
        vec3 position;
        vec3 color;
    }
    struct CoarseVertex {
        vec3 color;
    }
    struct Fragment {
        vec4 color;
    }
    struct VertexStageOutput {
        CoarseVertex coarseVertex;
        vec4 sv_position;
    }
    // Constant Buffers
    cbuffer Uniforms {
        mat4 modelViewProjection;
    }
vertex {
    VertexStageOutput vertexMain(AssembledVertex  assembledVertex) {
        VertexStageOutput output;
        vec3 position = assembledVertex.position;
        vec3 color = assembledVertex.color;
        coarseVertex.color =  color;
        sv_position = mul( modelViewProjection, vec4( position, 1.0));
        return  output;
    }

}
fragment {
    vec4 fragmentMain(CoarseVertex  coarseVertex) {
        vec3 color = coarseVertex.color;
        return vec4( color, 1.0);
    }

}
}



### The Power of Two-Way Translation 💪

This bidirectional translation capability offers several key advantages:

1. **Legacy Code Integration**: Easily incorporate existing shaders from various platforms into your CrossGL workflow.
2. **Migration Assistance**: Smoothly transition projects from platform-specific implementations to the universal CrossGL standard.
3. **Learning Tool**: Use the translator to understand how platform-specific constructs map to CrossGL, aiding in the learning process.

By providing this two-way translation, CrossGL establishes itself as a comprehensive solution for cross-platform shader development, bridging the gap between different graphics APIs and simplifying the shader writing process.

## 🎉 Conclusion: The Full Circle of Shader Development

We've now seen the complete cycle of CrossGL's capabilities:
1. Writing shaders in the universal CrossGL language
2. Translating CrossGL to platform-specific shaders (Metal, DirectX, OpenGL)
3. Converting existing platform-specific shaders back to CrossGL

This full-circle approach empowers developers to work seamlessly across different graphics APIs, promoting code reuse, simplifying multi-platform development, and accelerating the shader creation process.

With CrossGL, the future of graphics programming is not just cross-platform - it's boundaryless! 🚀✨
```

This addition to the notebook showcases CrossGL's ability to convert shaders from different graphics APIs back into its universal language, completing the circle of its translation capabilities. It provides concrete examples for each major platform and explains the benefits of this two-way translation feature.